## This is an basic implementation of naives bayes

In [1]:
# so fist thing is understanding what the standard bayes classifier is doing
import numpy as np

class BayesianClassifier:
    def __init__(self):


        # the class priors and likelihoods will be stored here
        # class_priors is a dictionary mapping class labels to their prior probabilities
        # for example, {0: 0.6, 1: 0.4} if there are two classes 0 and 1


        self.class_priors = {}

        # likelihoods is a dictionary mapping class labels to their likelihood parameters
        # the likelihoods parameters is the mean and variance for each feature given the class
        # so look at it like this:
        #       {0: {'mean': [mean1, mean2, ...], 
        #       'var': [var1, var2, ...]}, 1: {...}}

        # so the dictionary structure is like this
        # class_label -> {'mean': [...], 'var': [...]}
        # 0 or 1 -> {'mean' : [mean0, mean1], 'var': [var0, var1]}

        # where the mean of the class 0 and variance of class 0 are stored in lists
        # so to access it would something like this: 
        # self.likelihoods[0]['mean'] to get the mean vector for class 0
        # self.likelihoods[1]['var'] to get the variance vector for class 1

        self.likelihoods = {}


        # store the unique classes so in this case 0 and 1

        self.classes = []

    def fit(self, X, y):

        # get the unique classes

        self.classes = np.unique(y)

        # total number of samples

        total_samples = len(y)

        #
        for cls in self.classes:
            # so this gets all the classes in terms of same based on the y labels
            # so for some X_cls[i] is the list of samples that belong to class cls
            X_cls = X[y == cls]
            
            # the probability is the frequency of the class in the dataset
            self.class_priors[cls] = len(X_cls) / total_samples
            # and then we store the mean of each feature 
            self.likelihoods[cls] = {
                # X_cls is all the samples that belong class cls and 
                
                'mean': np.mean(X_cls, axis=0),# this will returns the mean of each class feature as a list
                'var': np.var(X_cls, axis=0)# this will return the variance of each class feature as a list
            }

    def _gaussian_likelihood(self, x, mean, var):
        # this in mathematical terms is the probability density function of a normal distribution
        # the probability density function is given by:
        # P(x|mean, var) = (1 / sqrt(2 * pi * var)) * exp(-((x - mean)^2) / (2 * var))
        exponent = np.exp(-((x - mean) ** 2) / (2 * var))
        normalization = (1 / np.sqrt(2 * np.pi * var))
        return normalization * exponent

    def predict(self, X):
        # list predictions will contain predicted class labels for each sample in X
        predictions = []
        for x in X:
            #
            class_probs = {}
            for cls in self.classes:
                # the prior are all the possible y values
                prior = self.class_priors[cls]
                # likehood per feature
                # so the gaussian likelihood returns the a list of likelihood of each feature
                likelihood = np.prod(self._gaussian_likelihood(x, 
                                                                self.likelihoods[cls]['mean'], 
                                                                self.likelihoods[cls]['var']))
                class_probs[cls] = prior * likelihood
            predictions.append(max(class_probs, key=class_probs.get))
        return np.array(predictions)
    
    # this is a better production ready version of predict based on the log probabilities
    def predict_log(self, X):
        predictions = []
        # so for each sample in X
        for x in X:
            class_log_prob = {}
            # get the probabiteis of each class
            for cls in self.classes:
                # so the log values make it easir to compute since instead of multplication we do 
                # addition and for extremly small values that would underflow become stable
                log_prior = np.log(self.class_priors[cls])
                # so the gausian likelihood of each feature given the class
                # this is a list of likelihood that in regards to some class variable y
                likelihoods = self._gaussian_likelihood(x,self.likelihoods[cls]['mean'], self.likelihoods[cls]['var'])
                log_likelihood = np.sum(np.log(likelihoods))
                class_log_prob[cls] = log_prior + log_likelihood
            predictions.append(max(class_log_prob, key=class_log_prob.get))
        return np.array(predictions)